In [1]:
import glob
import pandas as pd
import numpy as np

import scipy.sparse
import time
from sklearn import svm
import joblib
import sklearn
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, recall_score, precision_score, matthews_corrcoef, f1_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn import calibration, model_selection

In [2]:

import numpy as np
import scipy.sparse


def tanimotokernel(data_1, data_2):
    if isinstance(data_1, scipy.sparse.csr_matrix) and isinstance(data_2, scipy.sparse.csr_matrix):
        return _sparse_tanimotokernel(data_1, data_2)
    elif isinstance(data_1, scipy.sparse.csr_matrix) or isinstance(data_2, scipy.sparse.csr_matrix):
        # try to sparsify the input
        return _sparse_tanimotokernel(scipy.sparse.csr_matrix(data_1), scipy.sparse.csr_matrix(data_2))
    else:  # both are dense
        return _dense_tanimotokernel(data_1, data_2)
    
    
    
    
def _dense_tanimotokernel(data_1, data_2):
    """
    Tanimoto kernel
        K(x, y) = <x, y> / (||x||^2 + ||y||^2 - <x, y>)
    as defined in:
    "Graph Kernels for Chemical Informatics"
    Liva Ralaivola, Sanjay J. Swamidass, Hiroto Saigo and Pierre Baldi
    Neural Networks
    https://www.sciencedirect.com/science/article/pii/S0893608005001693
    http://members.cbio.mines-paristech.fr/~jvert/svn/bibli/local/Ralaivola2005Graph.pdf
    """

    norm_1 = (data_1 ** 2).sum(axis=1).reshape(data_1.shape[0], 1)
    norm_2 = (data_2 ** 2).sum(axis=1).reshape(data_2.shape[0], 1)
    prod = data_1.dot(data_2.T)

    divisor = (norm_1 + norm_2.T - prod) + np.finfo(data_1.dtype).eps
    return prod / divisor



def _sparse_tanimotokernel(data_1, data_2):
    """
    Tanimoto kernel
        K(x, y) = <x, y> / (||x||^2 + ||y||^2 - <x, y>)
    as defined in:
    "Graph Kernels for Chemical Informatics"
    Liva Ralaivola, Sanjay J. Swamidass, Hiroto Saigo and Pierre Baldi
    Neural Networks
    https://www.sciencedirect.com/science/article/pii/S0893608005001693
    http://members.cbio.mines-paristech.fr/~jvert/svn/bibli/local/Ralaivola2005Graph.pdf
    """

    norm_1 = np.array(data_1.power(2).sum(axis=1).reshape(data_1.shape[0], 1))
    norm_2 = np.array(data_2.power(2).sum(axis=1).reshape(data_2.shape[0], 1))
    prod = data_1.dot(data_2.T).A

    divisor = (norm_1 + norm_2.T - prod) + np.finfo(data_1.dtype).eps
    result = prod / divisor
    return result

def _minmaxkernel_numpy(data_1, data_2):
    """
    MinMax kernel
        K(x, y) = SUM_i min(x_i, y_i) / SUM_i max(x_i, y_i)
    bounded by [0,1] as defined in:
    "Graph Kernels for Chemical Informatics"
    Liva Ralaivola, Sanjay J. Swamidass, Hiroto Saigo and Pierre Baldi
    Neural Networks
    http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.92.483&rep=rep1&type=pdf
    """
    return np.stack([(np.minimum(data_1, data_2[cpd,:]).sum(axis=1) / np.maximum(data_1, data_2[cpd,:]).sum(axis=1))  for cpd in range(data_2.shape[0])],axis=1)


try: 
    import numba
    from numba import njit, prange
    
    @njit(parallel=True,fastmath=True)
    def _minmaxkernel_numba(data_1, data_2):
        """
        MinMax kernel
            K(x, y) = SUM_i min(x_i, y_i) / SUM_i max(x_i, y_i)
        bounded by [0,1] as defined in:
        "Graph Kernels for Chemical Informatics"
        Liva Ralaivola, Sanjay J. Swamidass, Hiroto Saigo and Pierre Baldi
        Neural Networks
        http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.92.483&rep=rep1&type=pdf
        """


        result = np.zeros((data_1.shape[0], data_2.shape[0]), dtype=np.float64)

        for i in prange(data_1.shape[0]):
            for j in prange(data_2.shape[0]):
                result[i,j] = _minmax_two_fp(data_1[i], data_2[j])
        return result


    @njit(fastmath=True)
    def _minmax_two_fp(fp1, fp2):
        common = numba.int32(0)
        maxnum = numba.int32(0)
        i = 0
        
        while i < len(fp1):
            min_ = fp1[i]
            max_ = fp2[i]

            if min_ > max_:
                min_ = fp2[i]
                max_ = fp1[i]

            common += min_
            maxnum += max_

            i += 1
            
        return numba.float64(common) / numba.float64(maxnum)

    minmaxkernel = _minmaxkernel_numba
    
except:
    
    print("Couldn't find numba. I suggest to install numba to compute the minmax kernel much much faster")
    
    minmaxkernel = _minmaxkernel_numpy



In [3]:
def load_data(target, random_state=None):
    df = pd.read_pickle("{}_df.pkl.gz".format(target))
    if random_state:
        df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    training = df[df.trainingset_class == "training"]
    test = df[df.trainingset_class == "test"]
    validation = df[df.trainingset_class == "validation"]

    training_X = np.array([np.array(e) for e in training.cfp.values],dtype=np.float64, order='C')
    training_Y = np.array(training.activity_label, dtype=np.float64, order='C')

    test_X = np.array([np.array(e) for e in test.cfp.values],dtype=np.float64, order='C')
    test_Y = np.array(test.activity_label, dtype=np.float64, order='C')

    validation_X = np.array([np.array(e) for e in validation.cfp.values],dtype=np.float64, order='C')
    validation_Y = np.array(validation.activity_label, dtype=np.float64, order='C')

    return training_X, training_Y, test_X, test_Y, validation_X, validation_Y

In [4]:
def parse_stats(file):    
    scores = {}
    with open(file) as fd:
        lines = fd.readlines()
        for line in lines:
            line = line.strip()
            splits = line.split("\t")
            if len(splits) == 3:
                set_, scorename, score = splits
            else:
                set_, scorename, threshold, score = splits
                scorename = f'{scorename} {threshold}'
                
            scorename = scorename[:-1] #remove the : sign
            score = float(score)
            if set_ not in scores:
                scores[set_] = {}
            scores[set_][scorename] = score
    return scores

In [5]:
for target in ['DRD2','HTR1A']:
    files = {}
    for file in glob.glob("models/{}*.stats".format(target)):
        files[file] = parse_stats(file)

    best_clf = None
    best_score = 0

    scorename = "F1"
    setname = 'Validation set'
    for key,value in files.items():
        if value[setname][scorename] > best_score:
            best_score = value[setname][scorename]
            best_clf = key

    
    target_name,c,cvalue,kernel,kernelname,class_weight,suffix = best_clf.split("_")
    
    cvalue = float(cvalue)
    
    if kernelname == 'tanimoto':
        kernel = tanimotokernel
    elif kernelname == 'minmax':
        kernel = minmaxkernel
    else:
        kernel = kernelname
        
    if class_weight == 'unbalanced':
        class_weight = None

    print(f'{"Name                                              ":30}\t{"ACC"}\t{"AUC"}\t{"F1"}\t{"MCC"}')
    setname = 'Training set'
    print(f'{best_clf:30}\t{files[best_clf][setname]["Balanced Accuracy"]:.2}\t{files[best_clf][setname]["ROC AUC"]:.4}\t{files[best_clf][setname]["F1"]:.2}\t{files[best_clf][setname]["MCC"]:.2}')
    setname = 'Validation set'
    print(f'{best_clf:30}\t{files[best_clf][setname]["Balanced Accuracy"]:.2}\t{files[best_clf][setname]["ROC AUC"]:.4}\t{files[best_clf][setname]["F1"]:.2}\t{files[best_clf][setname]["MCC"]:.2}')
    setname = 'Test set'
    print(f'{best_clf:30}\t{files[best_clf][setname]["Balanced Accuracy"]:.2}\t{files[best_clf][setname]["ROC AUC"]:.4}\t{files[best_clf][setname]["F1"]:.2}\t{files[best_clf][setname]["MCC"]:.2}')
    print()
    
    training_X, training_Y, test_X, test_Y, validation_X, validation_Y = load_data(target) 

    
    clf = svm.SVC(C=cvalue, random_state=1234, kernel=kernel, cache_size=1900 ,probability=True, class_weight=class_weight)
    print("Refit")
    clf.fit(training_X, training_Y)
    
    def score_clf(clf):
        sets = {"Training set": (training_X, training_Y) ,"Test set": (test_X, test_Y), "validation set": (validation_X, validation_Y) }
        scores_binary = {"Balanced Accuracy": lambda x,y: balanced_accuracy_score(x,y, adjusted=False), "Recall": recall_score, "Precision":  precision_score, "MCC": matthews_corrcoef, "F1": f1_score }
        scores_proba = {"ROC AUC": roc_auc_score }

        scores = {}
        for setname, data in sets.items():
            scores[setname] = {}
            data_X = data[0]
            data_Y = data[1]

            predicted_Y = clf.predict(data_X)
            predicted_Y_proba = clf.predict_proba(data_X)[:,1]

            for scorename, score_fn in scores_binary.items():
                scores[setname][scorename] = score_fn(data_Y, predicted_Y)

            for scorename, score_fn in scores_proba.items():
                scores[setname][scorename] = score_fn(data_Y, predicted_Y_proba)

        return scores
    
    clf.kernel = kernelname
    joblib.dump(clf, "{}_final.pkl".format(target), compress=("xz",9), protocol=-1)
    clf.kernel = kernel
    
#     print("Scoring: ...")
#     scores = score_clf(clf)
    
#     if scores.items() == files[best_clf].items():
#         print(" All metrics are the same.")
#     else:
#         print("Old Values: ")
#         print(scores.items())
#         print("New Values: ")
#         print(files[best_clf].items())
    
#     #calibrate the classifier using platt scaling
    
#     # we calibrate using the training and test set
#     train_test_X = np.concatenate([training_X,test_X], axis=0)
#     train_test_Y = np.concatenate([training_Y,test_Y], axis=0)
#     sample_weights_train_test = compute_sample_weight(class_weight, train_test_Y)
    
#     print("Calibrate")
#     clf_calibrated = calibration.CalibratedClassifierCV(base_estimator=clf, method='sigmoid', cv= 'prefit')
#     clf_calibrated.fit(train_test_X, train_test_Y, sample_weight=sample_weights_train_test)
        
#     for c in clf_calibrated.calibrated_classifiers_:
#         c.base_estimator.kernel = kernelname
#     joblib.dump(clf_calibrated, "/Users/thomas/projects/reinvent-classifiers/{}_final.pkl".format(target), compress=("xz",9), protocol=-1)
#     for c in clf_calibrated.calibrated_classifiers_:
#         c.base_estimator.kernel = kernel
    
#     def score_clf_print(clf, thresholds=[0.5]):
#         sets = {"Training set": (training_X, training_Y) ,"Test set": (test_X, test_Y), "validation set": (validation_X, validation_Y) }
#         scores_binary = {"Balanced Accuracy": lambda x,y: balanced_accuracy_score(x,y, adjusted=False), "Recall": recall_score, "Precision":  precision_score, "MCC": matthews_corrcoef, "F1": f1_score }
#         scores_proba = {"ROC AUC": roc_auc_score }

#         scores = {}
#         for setname, data in sets.items():
#             data_X = data[0]
#             data_Y = data[1]

#             predicted_Y_proba = clf.predict_proba(data_X)[:,1]
#             for threshold in thresholds:
#                 if threshold not in scores:
#                     scores[threshold] = {}


#                 predicted_Y = np.array(predicted_Y_proba > threshold, dtype=np.float)

#                 for scorename, score_fn in scores_binary.items():
#                     scores[threshold]["{}\t{}".format(setname, scorename)] = score_fn(data_Y, predicted_Y)

#                 for scorename, score_fn in scores_proba.items():
#                     scores[threshold]["{}\t{}".format(setname, scorename)] = score_fn(data_Y, predicted_Y_proba)

#         return scores

#     print("Scoring for print: ...")
#     scores_thres = score_clf_print(clf_calibrated, thresholds=[0.5,0.6,0.7,0.8,0.85,0.9])


#     for threshold, scores in scores_thres.items(): 
#         with open('/Users/thomas/projects/reinvent-classifiers/{}_final_calibratedcv.stats.{}'.format(target,threshold), "w") as fd:
#                 for scorename, score in scores.items():
#                     line = "{}:\t{}\n".format(scorename, score)
#                     fd.write(line)

    

Name                                              	ACC	AUC	F1	MCC
models/DRD2_c_0.1_kernel_minmax_balanced_proba.stats	0.99	0.998	0.77	0.79
models/DRD2_c_0.1_kernel_minmax_balanced_proba.stats	0.93	0.989	0.7	0.71
models/DRD2_c_0.1_kernel_minmax_balanced_proba.stats	0.96	0.9837	0.71	0.72

Refit
Name                                              	ACC	AUC	F1	MCC
models/HTR1A_c_0.1_kernel_minmax_balanced_proba.stats	0.98	0.9948	0.77	0.78
models/HTR1A_c_0.1_kernel_minmax_balanced_proba.stats	0.96	0.9877	0.74	0.75
models/HTR1A_c_0.1_kernel_minmax_balanced_proba.stats	0.96	0.9897	0.75	0.75

Refit


In [6]:
for target in ['DRD2','HTR1A']:   
    training_X, training_Y, test_X, test_Y, validation_X, validation_Y = load_data(target) 

    
    clf = joblib.load(f'{target}_final.pkl')
    
    if clf.kernel == 'tanimoto':
        clf.kernel = tanimotokernel
    elif clf.kernel == 'minmax':
        clf.kernel = minmaxkernel
    else:
        clf.kernel = clf.kernel
        
    
    def score_clf(clf):
        sets = {"Training set": (training_X, training_Y) ,"Test set": (test_X, test_Y), "Validation set": (validation_X, validation_Y) }
        scores_binary = {"Balanced Accuracy": lambda x,y: balanced_accuracy_score(x,y, adjusted=False), "Recall": recall_score, "Precision":  precision_score, "MCC": matthews_corrcoef, "F1": f1_score }
        scores_proba = {"ROC AUC": roc_auc_score }

        scores = {}
        for setname, data in sets.items():
            data_X = data[0]
            data_Y = data[1]
            
            predicted_Y = clf.predict(data_X)
            predicted_Y_proba = clf.predict_proba(data_X)[:,1]
            for scorename, score_fn in scores_binary.items():
                scores["{}\t{}".format(setname, scorename)] = score_fn(data_Y, predicted_Y)

            for scorename, score_fn in scores_proba.items():
                scores["{}\t{}".format(setname, scorename)] = score_fn(data_Y, predicted_Y_proba)

        return scores
    
    
    scores = score_clf(clf)
    
    with open(f'{target}_final.stats', "w") as fd:
            for scorename, score in scores.items():
                line = "{}:\t{}\n".format(scorename, score)
                fd.write(line)


    